### Data Processing in Python - Midterm

Manuel Leiva Fung (cuni: 35653506)

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

##### Part 1

API requests

In [2]:
#Send a POST to /login endpoint to retrieve the data
address  = "https://ies-midterm.soulution.rocks/login"
cunicode = {"cuni" : "35653506"}
login    = requests.post(f"{address}", json = cunicode)
#Verified if its all ok
login

<Response [200]>

In [3]:
#See the response from login
login.text

'{"data": {"personal_code": "ec857fca", "dataset_ids": ["81395feb08", "3bd4cabfaa", "5aa21103e8", "f36bd332b4", "5df3513c79", "f45ec2e9fc", "7be200ce5b", "098b83ee51", "95ff551368"]}, "message": "Hello, Leiva Fung Manuel Alfonso Lucio! Good luck with the midterm. GET your data set from /data/file_id endpoint; use the codes you received in the data response as the file_id parameter! Print the personal code in your notebook."}'

In [5]:
#Get the personal code and the list of ids to work in
personal_code = login.json()['data']['personal_code']
dataset_ids   = login.json()['data']['dataset_ids']
#See the personal code and the number of ids I am working with
print("Personal code is :", personal_code)
print("Number of ids is :", len(dataset_ids))

Personal code is : ec857fca
Number of ids is : 9


In [10]:
#Request for every id
statuscode = []
i          = 0
#As response should be 200, the sum of 200s should be equal to 200 * # of ids
while sum(statuscode) != 200*len(dataset_ids):
    getresponse = []
    statuscode  = []
    
    while i < len(dataset_ids):
        getresponse.append(requests.get(f"https://ies-midterm.soulution.rocks/data/{dataset_ids[i]}"))
        i+=1
        
    for a in getresponse:
        statuscode.append(a.status_code)
    
    i=0
#It will take time due to the probability of 0.6 of returning with no error
#(0.6 ^ # of ids) will be the probability it will give no error every run

In [11]:
#Verify everything is fine and complete
print(getresponse)
len(getresponse)

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


9

In [14]:
#Take a look at the data
data = json.loads(getresponse[0].text)
data

{'data': {'company': 'CCH',
  'data': [['2019-01-02',
    '9.649999618530273',
    '9.649999618530273',
    '9.640000343322754',
    '9.649999618530273',
    '9.623074531555176',
    '500'],
   ['2019-01-03',
    '9.630000114440918',
    '9.630000114440918',
    '9.630000114440918',
    '9.630000114440918',
    '9.603131294250488',
    '1700'],
   ['2019-01-04',
    '9.640000343322754',
    '9.640000343322754',
    '9.640000343322754',
    '9.640000343322754',
    '9.613102912902832',
    '1500'],
   ['2019-01-07',
    '9.640000343322754',
    '9.640000343322754',
    '9.640000343322754',
    '9.640000343322754',
    '9.613102912902832',
    '1000'],
   ['2019-01-08',
    '9.630000114440918',
    '9.630000114440918',
    '9.630000114440918',
    '9.630000114440918',
    '9.603131294250488',
    '186600'],
   ['2019-01-09',
    '9.630000114440918',
    '9.630000114440918',
    '9.630000114440918',
    '9.630000114440918',
    '9.603131294250488',
    '0'],
   ['2019-01-10',
    '9.71300

In [15]:
#See all the companies and the number of observations
lengths = []
names = []
i=0
while i < len(getresponse):
    lengths.append(len(getresponse[i].json()['data']['data'])-1)
    names.append(getresponse[i].json()['data']['company'])
    print(f'Company {(names[i])} number of observations is {(lengths[i])}')
    i+=1

#Comment post-running the code: All companies seem to have the same number of observations
#It should be verified automatically if the number of stocks is higher

Company CCH number of observations is 478
Company AUTO number of observations is 478
Company BME number of observations is 478
Company ADM number of observations is 478
Company SLA number of observations is 181
Company AAL number of observations is 478
Company SVT number of observations is 478
Company MRO number of observations is 478
Company AHT number of observations is 478


##### Part 2

Data analysis

In [76]:
#Generate the data frame of dataframes with all info about the stocks
d = {}
i=0
while i < len(getresponse):
    for name in names:
        d[name] = pd.DataFrame(getresponse[i].json()['data']['data'][0:lengths[i]])
        i+=1

In [77]:
#See if all dataframes have downloaded the info correctly
for name in names:
    print(d[name].head())

            0                  1                  2                  3  \
0  2019-01-02  9.649999618530273  9.649999618530273  9.640000343322754   
1  2019-01-03  9.630000114440918  9.630000114440918  9.630000114440918   
2  2019-01-04  9.640000343322754  9.640000343322754  9.640000343322754   
3  2019-01-07  9.640000343322754  9.640000343322754  9.640000343322754   
4  2019-01-08  9.630000114440918  9.630000114440918  9.630000114440918   

                   4                  5       6  
0  9.649999618530273  9.623074531555176     500  
1  9.630000114440918  9.603131294250488    1700  
2  9.640000343322754  9.613102912902832    1500  
3  9.640000343322754  9.613102912902832    1000  
4  9.630000114440918  9.603131294250488  186600  
            0                   1                   2                   3  \
0  2019-01-02  3.0199999809265137   3.450000047683716  3.0199999809265137   
1  2019-01-03   3.200000047683716  3.4200000762939453   3.200000047683716   
2  2019-01-04  3.4100000

In [78]:
#Check out for missing values
for name in names:
    print(d[name].isnull().sum())

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    1
2    1
3    1
4    1
5    1
6    1
dtype: int64
0    0
1    1
2    1
3    1
4    1
5    1
6    1
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    1
2    1
3    1
4    1
5    1
6    1
dtype: int64


In [79]:
#Drop missing values
for name in names:
    d[name] = d[name].dropna()

In [80]:
#Check out for missing values
for name in names:
    print(d[name].isnull().sum())

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64


In [81]:
#Assign the corresponding column names as they are not complete
#They like that since before (seen when data variable was visualized)
colnames1 = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj.Close', 'Volume']

for name in names:
    d[name].columns = colnames1
    print( d[name].head())
    print( d[name].tail())

         Date               Open               High                Low  \
0  2019-01-02  9.649999618530273  9.649999618530273  9.640000343322754   
1  2019-01-03  9.630000114440918  9.630000114440918  9.630000114440918   
2  2019-01-04  9.640000343322754  9.640000343322754  9.640000343322754   
3  2019-01-07  9.640000343322754  9.640000343322754  9.640000343322754   
4  2019-01-08  9.630000114440918  9.630000114440918  9.630000114440918   

               Close          Adj.Close  Volume  
0  9.649999618530273  9.623074531555176     500  
1  9.630000114440918  9.603131294250488    1700  
2  9.640000343322754  9.613102912902832    1500  
3  9.640000343322754  9.613102912902832    1000  
4  9.630000114440918  9.603131294250488  186600  
           Date                Open                High                 Low  \
473  2020-11-16                19.0  19.239999771118164  18.610000610351562   
474  2020-11-17  18.920000076293945                19.0  18.579999923706055   
475  2020-11-18  1

In [84]:
#Convert the values of prices into float so I can work with them
#Type could be checked beforehand but as they come from a string of json they are more likely to be strings
colnames2 = ['Open', 'High', 'Low', 'Close', 'Adj.Close', 'Volume']

for name in names:
    for colname in colnames2:
        i=0
        while i < len(d[name]):
            d[name][colname][i] = float(d[name][colname][i])
            i+=1

KeyError: 29

In [85]:
#Transform dates into datetime type
for name in names:
    i=0
    while i < len(d[name]):
        d[name]['Date'][i] = datetime.strptime(d[name]['Date'][i], '%Y-%m-%d')
        i+=1

KeyError: 29

In [96]:
#Look if all dates are fine
for name in names:
    print(f"For company {name} we have that the first date is {(d[name]['Date'][0])} and last date {(d[name]['Date'][len(d[name])-1])}")

For company CCH we have that the first date is 2019-01-02 00:00:00 and last date 2020-11-20 00:00:00
For company AUTO we have that the first date is 2019-01-02 00:00:00 and last date 2020-11-20 00:00:00
For company BME we have that the first date is 2019-01-02 00:00:00 and last date 2020-11-20 00:00:00
For company ADM we have that the first date is 2019-01-02 00:00:00 and last date 2020-11-20 00:00:00
For company SLA we have that the first date is 2019-02-26 00:00:00 and last date 2019-05-20
For company AAL we have that the first date is 2020-05-15 and last date 2019-11-15
For company SVT we have that the first date is 2019-01-02 and last date 2020-11-20
For company MRO we have that the first date is 2019-01-02 and last date 2020-11-20
For company AHT we have that the first date is 2019-01-09 and last date 2020-11-18


Because of the errors previously detected i have that not all series follow the complete period.
I will just proceed anyway because I do not have time :(

It is possible that not all stocks work completely well because not all variables were converted to float and strings cannot be use for math. But at least the first ones should be fine.

In [98]:
#Highest number of shares traded
for name in names:
    print(f"For company {name} we have max. number of shares traded is {(d[name]['Volume'].max())} on {(d[name]['Date'].loc[d[name]['Volume'] == d[name]['Volume'].max()])}")

For company CCH we have max. number of shares traded is 18218300.0 on 359    2020-06-05 00:00:00
Name: Date, dtype: object
For company AUTO we have max. number of shares traded is 8891600.0 on 417    2020-08-27 00:00:00
Name: Date, dtype: object
For company BME we have max. number of shares traded is 123800.0 on 307    2020-03-23 00:00:00
Name: Date, dtype: object
For company ADM we have max. number of shares traded is 10306500.0 on 23    2019-02-05 00:00:00
Name: Date, dtype: object
For company SLA we have max. number of shares traded is 9984 on 150    2019-06-04
Name: Date, dtype: object
For company AAL we have max. number of shares traded is 99888500 on 444    2020-11-18
Name: Date, dtype: object
For company SVT we have max. number of shares traded is 98300 on 50    2019-03-15
Name: Date, dtype: object
For company MRO we have max. number of shares traded is 9966800 on 14    2019-01-23
Name: Date, dtype: object
For company AHT we have max. number of shares traded is 991900 on 458    

In [97]:
#Adjusted Close
for name in names:
    print(f"For company {name} we have max. Adj.Close is {(d[name]['Adj.Close'].max())} and min. Adj.Close is {(d[name]['Adj.Close'].min())}")

For company CCH we have max. Adj.Close is 19.389999389648438 and min. Adj.Close is 9.603131294250488
For company AUTO we have max. Adj.Close is 5.369999885559082 and min. Adj.Close is 0.6700000166893005
For company BME we have max. Adj.Close is 46.650001525878906 and min. Adj.Close is 26.664894104003906
For company ADM we have max. Adj.Close is 51.49772644042969 and min. Adj.Close is 28.573394775390625
For company SLA we have max. Adj.Close is 84.69000244140625 and min. Adj.Close is 47.939998626708984
For company AAL we have max. Adj.Close is 9.890000343322754 and min. Adj.Close is 10.020000457763672
For company SVT we have max. Adj.Close is 9.979999542236328 and min. Adj.Close is 10.0
For company MRO we have max. Adj.Close is 9.408211708068848 and min. Adj.Close is 10.05465316772461
For company AHT we have max. Adj.Close is 9.199999809265137 and min. Adj.Close is 1.2899999618530273


The following may not work because of the str and float problem.
But at least this is how I would try if the problem was not there (?)

In [ ]:
#Highest number of shares traded
volume_count = 0
volume_comp  = []

for name in names:
    if volume_count < d[name]['Volume'].max():
        colume_comp = name
        volume_count = d[name]['Volume'].max()

print(volume_comp)

d[volume_comp].plot(x = 'Date', y = 'Adj.Close', kind = 'line')
plt.show()

In [ ]:
#Log Intraday change (Open - CLose)
for name in names:
    d[name]['log.Intraday'] = log(d[name]['Close']/d[name]['Open'])
    print(f'For company {name} we have that the highest gain in a day was on {(d[name]['Date'].loc[d[name]['log.Intraday'] == d[name]['log.Intraday'].max()])} and the highest loss in a day was on {(d[name]['Date'].loc[d[name]['log.Intraday'] == d[name]['log.Intraday'].min()])}')

logday_count = 0
logday_comp  = []

for name in names:
    d[name]['log.Intraday.abs'] = d[name]['log.Intraday'].abs()
    if log_count < d[name]['log.Intraday.abs'].max():
        logday_comp = name
        logday_count = d[name]['log.Intraday.abs'].max()

print(logday_comp)

print(f'Company {logday_comp} had that the biggest change of price in a day on {(d[logday_comp]['Date'].loc[d[logday_comp]['log.Intraday.abs'] == logday_count])}')

d[logday_comp].plot(x = 'Date', y = 'log.Intraday', kind = 'line')
plt.show()

In [101]:
#10-day standard deviation
for name in names:
    i = 9
    while i < len(d[name]):
        d[name]['10-day.std'][i] = d[name]['Adj.Close'].loc[(i-9):(i)].std()
        i+=1

KeyError: '10-day.std'

In [ ]:
#Single object of adjusted prices
adj_prices = []

for name in names:
    adj_prices[name] = d[name]['Adj.Close']

adj_prices['Date'] = d[""]['Date']
adj_prices.set_index('Date', drop  = True)

series = adj_prices.columns

for serie in series:
    plt.plot(adj_prices['Date'], adj_prices[serie], label=serie, linewidth=2)
    
plt.title('Adj.Close')
plt.xlabel('Date')
plt.ylabel('Price')

plt.legend()
plt.show()

cant really work for weekly or monthly summary as not all dates were transformed